<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import warnings

warnings.filterwarnings('ignore')

DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [64]:
query_3_1 = f'''SELECT COUNT(*)
                FROM public.VACANCIES
            '''

df = pd.read_sql_query(query_3_1, connection)
df

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [65]:
query_3_2 = f'''SELECT DISTINCT COUNT(id)
                FROM public.EMPLOYERS
            '''

df = pd.read_sql_query(query_3_2, connection)
df

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [66]:
query_3_3 = f'''SELECT DISTINCT COUNT(id)
                FROM public.AREAS
            '''

df = pd.read_sql_query(query_3_3, connection)
df

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [67]:
query_3_4 = f'''SELECT DISTINCT COUNT(id)
                FROM public.INDUSTRIES
            '''

df = pd.read_sql_query(query_3_4, connection)
df

,count
0,294


***

Что можно сказать исходя из предварительного анализа? Можно сказать следующее:
* в среднем каждый работодатель размещает 2 вакансии.
* в каждом регионе в среднем 36 вакансий.
* в среднем по каждой сфере деятельности открыто около 170 вакансий.

Очевидно, что эти цифры имеют крайне слабое отношение к реальности и данные требуют детального анализа.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [68]:
query_4_1 = f'''SELECT 
                    a.name,
                    COUNT(v.id)
                FROM public.VACANCIES AS v
                    JOIN public.AREAS AS a ON a.id = v.area_id
                GROUP BY
                    a.name
                ORDER BY 2 DESC
            '''

df = pd.read_sql_query(query_4_1, connection)
df.head(20)

,name,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


In [69]:
# считаем сумму вакансий первых 20-ти регионов
df[df.index < 20]['count'].sum()

29787

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [70]:
query_4_2 = f'''SELECT 
                    COUNT(v.id)
                FROM public.VACANCIES AS v
                WHERE
                    v.salary_from IS NOT NULL
                    OR v.salary_to IS NOT NULL          
            '''

df = pd.read_sql_query(query_4_2, connection)
df

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [71]:
query_4_3 = f'''SELECT 
                    AVG(v.salary_from) AS s_from,
                    AVG(v.salary_to) AS s_to
                FROM public.VACANCIES AS v       
            '''

df = pd.read_sql_query(query_4_3, connection)
round(df,0)

,s_from,s_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [72]:
query_4_4 = f'''SELECT 
                    v.schedule,
                    v.employment,
                    COUNT(v.id) AS c
                FROM public.VACANCIES AS v
                GROUP BY
                    1, 2
                ORDER BY c DESC
            '''

df = pd.read_sql_query(query_4_4, connection)
df

,schedule,employment,c
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [73]:
query_4_5 = f'''SELECT 
                    v.experience AS exp,
                    COUNT(v.id) AS c
                FROM public.VACANCIES AS v
                GROUP BY exp
                ORDER BY c
            '''

df = pd.read_sql_query(query_4_5, connection)
df

,exp,c
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Какие основные выводы из полученных данных мы можем сделать:

* в большинстве своём вакансии находятся в крупных городах, что очевидно.
* только в половине вакансий указана ЗП.
* работодатели заинтересованы прежде всего в работниках полного дня, и предпочитают чтобы работник находился в офисе.
* мода требуемого опыта - от 1 до 3 лет.

Уже лучше. На основе полученных данных можно "нарисовать" условного требуемого кандидата: предпочтительна полная занятость и работа в офисе, опыт от 1 до 6 лет, житель крупного города (Россия и ближнее зарубежье) или готов к переезду, заработная плата скорее определяется по результатам собеседования.

P.S. Данные по зарплатной вилке крайне слабо отражает реальное положение дел, если мы берём среднее по всем вакансиям и регионам.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [74]:
query_5_1 = f'''SELECT 
                    e.name,
                    COUNT(v.id) AS c
                FROM public.VACANCIES AS v
                LEFT JOIN
                    public.EMPLOYERS AS e ON e.id = v.employer_id
                GROUP BY 1
                ORDER BY 2 DESC
            '''

df = pd.read_sql_query(query_5_1, connection)
df.head(20)

,name,c
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
5,ИК СИБИНТЕК,327
6,МТС,292
7,DataArt,247
8,Совкомбанк Технологии,204
9,Первый Бит,176


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [75]:
query_5_2 = f'''SELECT 
                    a.name,
                    COUNT(v.id) AS cnt_v,
                    COUNT(e.id) AS cnt_e
                FROM public.AREAS AS a
                    LEFT JOIN public.EMPLOYERS AS e ON e.area = a.id
                    LEFT JOIN public.VACANCIES AS v ON v.area_id = a.id AND v.employer_id = e.id
                GROUP BY a.name
                ORDER BY
                    cnt_v,
                    cnt_e DESC
            '''

df = pd.read_sql_query(query_5_2, connection)
df

,name,cnt_v,cnt_e
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Великобритания,0,23
4,Краснодарский край,0,19
...,...,...,...
1357,Екатеринбург,960,1166
1358,Алматы,1069,1276
1359,Санкт-Петербург,1811,3069
1360,Минск,1858,2228


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [76]:
query_5_3 = f'''SELECT 
                    e.name,
                    COUNT(DISTINCT v.area_id) AS cnt_areas
                FROM public.EMPLOYERS AS e
                    JOIN public.VACANCIES AS v ON v.employer_id = e.id
                GROUP BY e.name
                ORDER BY
                    cnt_areas DESC
            '''

df = pd.read_sql_query(query_5_3, connection)
df.head(20)

,name,cnt_areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [77]:
query_5_4 = f'''SELECT 
                    COUNT(DISTINCT e.id)
                FROM public.EMPLOYERS AS e
                    LEFT JOIN public.EMPLOYERS_INDUSTRIES AS ei ON ei.employer_id = e.id
                    LEFT JOIN public.INDUSTRIES AS i ON i.id = ei.industry_id 
                WHERE 
                    i.id IS NULL                
            '''

df = pd.read_sql_query(query_5_4, connection)
df

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [78]:
query_5_5 = f'''SELECT
                    e.name
                FROM public.EMPLOYERS AS e
                    JOIN public.EMPLOYERS_INDUSTRIES AS ei ON ei.employer_id = e.id
                    JOIN public.INDUSTRIES AS i ON i.id = ei.industry_id 
                GROUP BY
                    e.name
                HAVING
                    COUNT(i.id) = 4
                ORDER BY
                    1
            '''

df = pd.read_sql_query(query_5_5, connection)
df.head(20)

,name
0,101 Интернет
1,21vek.by
2,2ГИС
3,2К
4,4 пикселя +
5,5lb
6,5 Армия
7,741 Studios
8,AAEngineering Group
9,ADAPTER


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [79]:
query_5_6 = f'''SELECT
                    COUNT(e.name)
                FROM public.EMPLOYERS AS e
                    JOIN public.EMPLOYERS_INDUSTRIES AS ei ON ei.employer_id = e.id
                    JOIN public.INDUSTRIES AS i ON i.id = ei.industry_id 
                WHERE
                    i.name = 'Разработка программного обеспечения'
            '''

df = pd.read_sql_query(query_5_6, connection)
df

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [80]:
# получаем список городов-миллионников

import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)

page = BeautifulSoup(response.text, 'html.parser')
links = page.find_all('a')[46:63]

s = []

for elem in links:
    start = 0
    stop = 0
    start = str(elem).find('title=\"')
    stop = str(elem).find('\">')
    if start > 0:
        s.append(str(elem)[start + 7:stop])

s = tuple(s)

In [81]:
query_5_7 = f'''WITH T AS(
                    SELECT
                        a.name AS name,
                        COUNT(v.id) AS cnt
                    FROM public.VACANCIES AS v
                        JOIN public.AREAS AS a ON a.id = v.area_id
                        JOIN public.EMPLOYERS AS e ON e.id = v.employer_id
                    WHERE
                        e.name = 'Яндекс'
                        AND a.name IN {s}
                    GROUP BY 1
                    ORDER BY 2 DESC)
                SELECT
                    name,
                    cnt
                FROM T
                UNION
                SELECT
                    'Total',
                    SUM(cnt)::int
                FROM T
                ORDER BY 2 DESC
            '''

df = pd.read_sql_query(query_5_7, connection)
df

,name,cnt
0,Total,353
1,Москва,54
2,Санкт-Петербург,42
3,Екатеринбург,39
4,Нижний Новгород,36
5,Новосибирск,35
6,Самара,26
7,Уфа,26
8,Ростов-на-Дону,25
9,Казань,25


***

Из полученной информации можно сделать следующие выводы:
* в лидерах по числу вакансий - компании, так или иначе существенно связанные с информационными технологиями, за некоторыми исключениями.
* при оценке количества вакансий в регионах для конкретного работодателя мы видим практически ту же картину.
* треть работодателей не указывают свой род деятельности.
* 15% работодателей имеют сферу деятельности "разработка программного обеспечения" (непонятно, много это или мало).

Очевидно, что не все вакансии однозначно относятся к сфере деятельности компании, например секретарь или водитель в банке. Верно также и обратное.

Нужен более детальный анализ.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [82]:
query_6_1 = f'''SELECT
                    COUNT(v.id)
                FROM public.VACANCIES AS v
                WHERE
                    LOWER(v.name) LIKE '%данн%'
                    OR LOWER(v.name) LIKE '%data%'
            '''

df = pd.read_sql_query(query_6_1, connection)
df

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [83]:
query_6_2 = f'''WITH vc AS(
                    SELECT 
                        v.id v_id,
                        v.name v_name,
                        v.experience v_exp,
                        v.employment v_emp
                    FROM public.VACANCIES AS v
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                )
                SELECT 
                    COUNT(v_id)
                FROM vc
                WHERE
                    LOWER(v_name) LIKE '%junior%'
                    OR v_exp LIKE '%Нет%опыт%'
                    OR v_emp LIKE 'Стажировка'
            '''

df = pd.read_sql_query(query_6_2, connection)
df

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [84]:
query_6_3 = f'''WITH vc AS(
                    SELECT 
                        v.id v_id,
                        v.name v_name,
                        v.experience v_exp,
                        v.employment v_emp,
                        v.key_skills v_key
                    FROM public.VACANCIES AS v
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                )
                SELECT 
                    COUNT(v_id)
                FROM vc
                WHERE
                    v_key ILIKE '%SQL%'
                    OR v_key ILIKE '%postgres%'
            '''

df = pd.read_sql_query(query_6_3, connection)
df

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [85]:
query_6_4 = f'''WITH vc AS(
                    SELECT 
                        v.id v_cnt,
                        v.name v_name,
                        v.experience v_exp,
                        v.employment v_emp,
                        v.key_skills v_key
                    FROM public.VACANCIES AS v
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                )
                SELECT 
                    COUNT(v_cnt)
                FROM vc
                WHERE
                    v_key ILIKE '%python%'
            '''

df = pd.read_sql_query(query_6_4, connection)
df

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [86]:
query_6_5 = f'''WITH vc AS(
                    SELECT 
                        v.id v_cnt,
                        v.name v_name,
                        v.experience v_exp,
                        v.employment v_emp,
                        v.key_skills v_key
                    FROM public.VACANCIES AS v
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                )
                SELECT
                    AVG(LENGTH(v_key) - LENGTH(REPLACE(v_key, '\t', '')) + 1) AS "Кол-во ключевых навыков"
                FROM vc
            '''

df = pd.read_sql_query(query_6_5, connection)
round(df, 2)

,Кол-во ключевых навыков
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [87]:
query_6_6 = f'''WITH vc AS(
                    SELECT 
                        v.experience v_exp,
                        v.salary_from v_sal_from,
                        v.salary_to v_sal_to,
                        COALESCE(v.salary_from, 0) / COALESCE(v.salary_from, 1) AS v_sal_from_i, --возвращает 0, если NULL, и 1 если иначе
                        COALESCE(v.salary_to, 0) / COALESCE(v.salary_to, 1) AS v_sal_to_i 
                    FROM public.VACANCIES AS v
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                )
                SELECT
                    v_exp AS "Опыт",
                    AVG((COALESCE(v_sal_from, 0) + COALESCE(v_sal_to, 0)) / (v_sal_from_i + v_sal_to_i)) AS "Средняя ЗП"
                FROM vc
                WHERE
                    v_sal_from IS NOT NULL
                    OR v_sal_to IS NOT NULL
                GROUP BY
                    v_exp         
            '''

df = pd.read_sql_query(query_6_6, connection)
round(df, 0)

,Опыт,Средняя ЗП
0,От 3 до 6 лет,243115.0
1,От 1 года до 3 лет,139675.0
2,Нет опыта,74643.0


***

Что можно сказать по полученным данным?
* около 3% вакансий имеют отношение к данным (полагаю, что эти вакансии не только для Data Scinetist-ов).
* мизерное относительно всех вакансий (около 0.1%) количество позиций для начинающего специалиста в Data Science.
* приличное среднее количество ключевых навыков - 7 (правда, сложно сказать, много это или мало).

По итогу можно сказать, что успешного соискателся ждёт неплохая зарплата :)

Честно говоря, на мой взгляд данных всё ещё очень недостаточно.

# Общий вывод по проекту

Вакансий для Data Scientist-а довольно мало, всего несколько сотен, поэтому нам очень важен порядок вывода для соискателя. Какие же алгоритмы можно задать для нашей модели?
Попробую предложить следующее:
* в первую очередь выводить вакансии непосредственно относящиеся к Data Science (очевидно).
* в выводе предпочтение отдавать компаниям с большим количеством вакансий и представленностью в регионах.
* при подборе вакансии следует учитывать сферу деятельности работодателя, но она не должна быть определяющей.
* сначала выводить вакансии для региона, где находится соискатель (очевидно), только если соискатель не указал удалённую работу как желаемую.
* модель должна сопоставлять опыт соискателя и требования работодателя и выводить наиболее релевантные вакансии.

Тем не менее, хочется получить больше информации из наших данных.

Для начала узнаем, сколько вообще вакансий непосредственно для Data Scientist-a:

In [88]:
query_7_1 = f'''WITH vc AS(
                    SELECT 
                        v.id v_id,
                        v.name v_name,
                        v.experience v_exp,
                        v.employment v_emp
                    FROM public.VACANCIES AS v
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                )
                SELECT 
                    COUNT(v_id)
                FROM vc
            '''

df = pd.read_sql_query(query_7_1, connection)
df

,count
0,483


Не густо - всего 1% от всех вакансий. Из них около 10% - для начинающего специалиста по DS.

Давайте узнаем, сколько вакансий для Data Scientist-ы с условием удалённой работы:

In [89]:
query_7_2 = f'''WITH vc AS(
                    SELECT 
                        v.id v_id,
                        v.name v_name,
                        v.experience v_exp,
                        v.employment v_emp
                    FROM public.VACANCIES AS v
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                        AND v.schedule ILIKE '%удален%'
                )
                SELECT 
                    COUNT(v_id)
                FROM vc
            '''

df = pd.read_sql_query(query_7_2, connection)
df

,count
0,413


Ого, больше 80%. Можно не ориентироваться на регион, где расположен работодатель, но иметь в виду. Тем более, что часть соискателей будет против удалённой работы.

Для порядка посмотрим распределение вакансий для DS по регионам:

In [100]:
query_7_3 = f'''WITH vc AS(
                    SELECT 
                        v.id v_id,
                        v.name v_name,
                        v.experience v_exp,
                        v.employment v_emp,
                        a.name v_area_name
                    FROM public.VACANCIES AS v
                    LEFT JOIN
                        public.AREAS as a ON a.id = v.area_id
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                )
                SELECT 
                    v_area_name,
                    COUNT(v_id)
                FROM vc
                GROUP BY v_area_name
                ORDER BY 2 DESC
            '''

df = pd.read_sql_query(query_7_3, connection)
df

,v_area_name,count
0,Москва,219
1,Санкт-Петербург,65
2,Новосибирск,23
3,Нижний Новгород,20
4,Алматы,16
5,Казань,16
6,Минск,9
7,Томск,8
8,Екатеринбург,7
9,Нур-Султан,6


Больше половины вакансий - Москва и Санкт-Петербург. Ради интереса посмотрим, сколько всего вакансий в городах-миллионниках России.

In [101]:
query_7_4 = f'''WITH vc AS(
                    SELECT 
                        v.id v_id,
                        v.name v_name,
                        v.experience v_exp,
                        v.employment v_emp,
                        a.name v_area_name
                    FROM public.VACANCIES AS v
                    LEFT JOIN
                        public.AREAS as a ON a.id = v.area_id
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                )
                SELECT 
                    COUNT(v_id)
                FROM vc
                WHERE v_area_name IN {s}
            '''

df = pd.read_sql_query(query_7_4, connection)
df

,count
0,365


Три четверти вакансий приходятся на города-миллионники.

Теперь посмотрим распределение вакансий по опыту:

In [102]:
query_7_5 = f'''WITH vc AS(
                    SELECT 
                        v.id v_id,
                        v.name v_name,
                        v.experience v_exp,
                        v.employment v_emp
                    FROM public.VACANCIES AS v
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                )
                SELECT 
                    v_exp,
                    COUNT(v_id)
                FROM vc
                GROUP BY 1
                ORDER BY 2 DESC
            '''

df = pd.read_sql_query(query_7_5, connection)
df

,v_exp,count
0,От 1 года до 3 лет,226
1,От 3 до 6 лет,195
2,Более 6 лет,34
3,Нет опыта,28


Мода опыта для DS совпадает с модой опыта по всем вакансиям. Эх, тяжело начинающему специалисту :(

Теперь посмотрим количество вакансий для DS, где не указана сфера деятельности компании:

In [103]:
query_7_6 = f'''WITH vc AS(
                    SELECT 
                        v.id v_id,
                        v.name v_name,
                        v.experience v_exp,
                        v.employment v_emp,
                        i.name v_ind_name
                    FROM public.VACANCIES AS v
                    LEFT JOIN public.EMPLOYERS_INDUSTRIES ei ON ei.employer_id = v.employer_id
                    LEFT JOIN public.INDUSTRIES i ON i.id = ei.industry_id
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                )
                SELECT 
                    COUNT(v_id)
                FROM vc
                WHERE v_ind_name IS NULL
            '''

df = pd.read_sql_query(query_7_6, connection)
df

,count
0,63


Хм, около 13%, что довольно мало. Стоит узнать, какие сферы деятельности у компаний с вакансиями для Data Scientist-а:

In [104]:
query_7_7 = f'''WITH vc AS(
                    SELECT 
                        v.id v_id,
                        v.name v_name,
                        v.experience v_exp,
                        v.employment v_emp,
                        i.name v_ind_name
                    FROM public.VACANCIES AS v
                    LEFT JOIN public.EMPLOYERS_INDUSTRIES ei ON ei.employer_id = v.employer_id
                    LEFT JOIN public.INDUSTRIES i ON i.id = ei.industry_id
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                )
                SELECT 
                    v_ind_name,
                    COUNT(v_id)
                FROM vc
                WHERE v_ind_name IS NOT NULL
                GROUP BY 1
                ORDER BY 2 DESC
            '''

df = pd.read_sql_query(query_7_7, connection)
df

,v_ind_name,count
0,Разработка программного обеспечения,148
1,"Системная интеграция, автоматизации технологи...",139
2,Банк,93
3,"Интернет-компания (поисковики, платежные систе...",79
4,Мобильная связь,23
...,...,...
92,"Добыча и обогащение минерального сырья (соль, ...",1
93,"Повышение квалификации, переквалификация",1
94,Водоснабжение и канализация,1
95,"Бетон, кирпич, стекло и прочие силикаты (произ...",1


In [105]:
# выведем топ-10 сфер деятельности
df.head(10)

,v_ind_name,count
0,Разработка программного обеспечения,148
1,"Системная интеграция, автоматизации технологи...",139
2,Банк,93
3,"Интернет-компания (поисковики, платежные систе...",79
4,Мобильная связь,23
5,"Маркетинговые, рекламные, BTL, дизайнерские, E...",22
6,"Управляющая, инвестиционная компания (управлен...",21
7,Консалтинговые услуги,16
8,Фиксированная связь,13
9,"Научно-исследовательская, научная, академическ...",9


Наше предположение, что сфера деятельности не слишком важна для DS, разбилось о суровую реальность. В модели однозначно стоит её учитывать!

Последнее, что мне хотелось бы узнать, какие ключевые навыки лидируют в вакансиях DS:

In [386]:
query_7_8 = f'''WITH vc AS(
                    SELECT 
                        v.id v_id,
                        v.name v_name,
                        v.experience v_exp,
                        v.employment v_emp,
                        v.key_skills v_key
                    FROM public.VACANCIES AS v
                    WHERE
                        LOWER(v.name) LIKE '%data%scientist%'
                        OR LOWER(v.name) LIKE '%data%science%'
                        OR LOWER(v.name) LIKE '%исследователь%данных%'
                        OR LOWER(v.name) LIKE '%machine%learning%'
                        OR LOWER(v.name) LIKE '%машинн%обучен%'
                        OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                )
                SELECT 
                    v_key,
                    COUNT(v_id)
                FROM vc
                WHERE v_key IS NOT NULL
                GROUP BY 1
                ORDER BY 2 DESC
            '''

df = pd.read_sql_query(query_7_8, connection)

# преобразуем элементы столбца ключевых навыков в список
df['v_key'] = df['v_key'].str.split('\t')

# используем счётчик для подсчёта ключевых навыков
from collections import Counter

key_skill_cnt = Counter()
for i in range(0, df.shape[0]):
    for elem in df['v_key'][i]:
        key_skill_cnt[elem] += df['count'][i]

# преобразуем полученный счётчик в DataFrame и сменим название столбцов
df_keys = pd.DataFrame.from_dict(key_skill_cnt, orient='index').reset_index()
df_keys.columns = ['Ключевой навык', 'Количество']

Полученный датафрейм не совсем корректен. Например: Machine Learning, ML, Машинное обучение это одно и то же. Сложим совпадающие строки и удалим лишнее.

P.S. Мы не будем править датафрейм полностью, исправим только самую значимую(?) часть - машинное обучение и аназлиз данных. Думаю, вернее надеюсь, что это не сильно исказит картину :)

In [387]:
ML_list = ['ML', 'Машинное обучение', 'machine learning']
An_list = ['Анализ данных']

for i in range(0, df.shape[0]):
    if df_keys.loc[i, 'Ключевой навык'] in ML_list:
        df_keys.loc[df_keys['Ключевой навык'] == 'Machine Learning', 'Количество'] += df_keys.loc[i, 'Количество']
        df_keys = df_keys.drop(labels=i)
    elif df_keys.loc[i, 'Ключевой навык'] in An_list:
        df_keys.loc[df_keys['Ключевой навык'] == 'Data Analysis', 'Количество'] += df_keys.loc[i, 'Количество']
        df_keys = df_keys.drop(labels=i)

# отсортируем по убыванию и сбросим индексы
df_keys = df_keys.sort_values(by='Количество', ascending=False)
df_keys = df_keys.reset_index()
df_keys = df_keys.drop(columns='index')

# мы получили датафрейм с ключевыми навыками и для каждого навыка количество вакансий, в которых он упоминается
# общее вакансий для DS равно 483. добавим столбец с относительным кол-вом вакансий для навыка
df_keys['%'] = round(df_keys['Количество'] * (100 / 483), 2)

# обрежем полученный датафрейм по топ-10 навыков
df_keys = df_keys.truncate(after=10)

In [390]:
# построим график для наглядности
import plotly.express as px

fig = px.bar(
    data_frame=df_keys,
    x = 'Ключевой навык',
    y = '%',
    color = 'Ключевой навык',
    title='Доля ключевого навыка по вакансиям Data Science'
)
fig.update_layout(
    title={
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    showlegend = False
)
fig.write_html("plotly/graph_keys.html")
fig.show();

Что можно сказать из полученного графика? Основные навыки для DS это:
* Python
* Машинное обучение
* SQL
* Анализ данных

Можно попробовать увязать ключевые навыки и опыт: чем меньше опыт, тем большее значение имеют базовые компетенции и вывод вакансий для начинающего специалиста будет основываться именно на базовых компетенциях. Также можно попробовать увязать ключевые навыки с желаемой зарплатой.

Какие ещё интересные доп. исследования можно сделать:
- среднюю по нижней и верхней гранцие зарплатной вилки для DS (в данном исследовании мы такое делали, но по всем вакансиям).
- тип рабочего графика и тип трудоустройства для DS (можно по регионам). Например, если в городе нет вакансий для DS, предлагать соискателям удалённую работу или вакансии в ближайших регионах.
- исследование работодателей: например, доля вакансий для DS из всех вакансий компании, и выводить соискателям в первую очередь компании с наибольшей долей.

***


Хотелось бы добавить несколько слов по поводу этого проекта.

Изначально наша цель была заявлена как: "создание модели машинного обучения, которая будет рекомендовать вакансии клиентам агентства, претендующим на позицию Data Scientist". И тут же мы имеем следующие вопросы:
- Каких вакансий много?
- Что можно сказать о рынке труда в различных городах?
- Какие ещё интересные исследования можно провести?

И если последние два вопроса ещё можно связать с нашей целью, то первый вопрос никаким боком не имеет отношения к поставленной задаче.

Дальше - больше: "Сделайте выводы не только на основе данных: попробуйте также проанализировать, за счёт чего получились именно такие цифры, постарайтесь увидеть тенденции, сделать прогнозы"

Почему у нас около 500 вакансий для DS? Как на это можно ответить? Как можно сделать какой-то прогноз или увидеть тенденцию, если мы имеем базу данных ХХ на какой-то момент времени. Ни динамики, ни какого-то события, которое как-то влияет на рынок труда.

P.S. Вполне возможно, что я чего-то не понимаю, прошу прокомментировать :)

